In [1]:
import pandas as pd
import numpy as np
import nltk
import re
import string
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer,HashingVectorizer

In [2]:
df = pd.read_csv("D:/PyProject/train.csv")

In [3]:
df.head(10)

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi...",0
6,6,Life: Life Of Luxury: Elton John’s 6 Favorite ...,NaN,Ever wonder how Britain’s most iconic pop pian...,1
7,7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi...",0
8,8,Excerpts From a Draft Script for Donald Trump’...,NaN,Donald J. Trump is scheduled to make a highly ...,0
9,9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...,0


In [5]:
#Get Independent Feature
X=df.drop('label',axis=1)

In [6]:
#Get Dependent Feature
y=df['label']

In [8]:
df.shape

(20800, 5)

In [7]:
#Remove null rows
df = df.dropna()

In [8]:
messages = df.copy()
messages.reset_index(inplace=True)

In [12]:
import nltk
stopwords = nltk.corpus.stopwords.words('english')
ps = PorterStemmer()
corpus1=[]

for i in range(0,len(messages)):
    review = re.sub('[^a-zA-Z0-9]',' ',messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review=[ps.stem(word) for word in review if not word in stopwords]
    review=' '.join(review)       
    corpus1.append(review)

In [18]:
#Applying CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
#Maximum top 5000 features with combination of 1,2 and 3 words
tfidf_V = TfidfVectorizer(max_features=5000,ngram_range=(1,3))
X = tfidf_V.fit_transform(corpus1).toarray()

In [19]:
X.shape

(18285, 5000)

In [20]:
y=messages['label']

In [23]:
#Divide the dataset into training and testing
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25,random_state=0)

In [22]:
tfidf_V.get__feature_names()[:20]

AttributeError: 'TfidfVectorizer' object has no attribute 'get__feature_names'

In [24]:
#Multinomial /nb
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()

In [15]:
#Metrics
from sklearn import metrics
from sklearn.metrics import confusion_matrix
import itertools

In [36]:
import matplotlib.pyplot as plt
import numpy as np
import itertools

accuracy = np.trace(cm) / np.sum(cm).astype('float')
misclass = 1 - accuracy
if cmap is None:
    cmap = plt.get_cmap('Blues')

plt.figure(figsize=(8, 6))
plt.imshow(cm, interpolation='nearest', cmap=cmap)
plt.title(title)
plt.colorbar()

if target_names is not None:
    tick_marks = np.arange(len(target_names))
    plt.xticks(tick_marks, target_names, rotation=45)
    plt.yticks(tick_marks, target_names)
if normalize:
    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

thresh = cm.max() / 1.5 if normalize else cm.max() / 2
for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    if normalize:
        plt.text(j, i, "{:0.4f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
    else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")

plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, misclass))
plt.show()


NameError: name 'cmap' is not defined

In [25]:
#PAssiveAgressive Classifier
from sklearn.linear_model import PassiveAggressiveClassifier
linear_clf = PassiveAggressiveClassifier(n_iter=50)

In [26]:
linear_clf.fit(X_train,y_train)
predict = linear_clf.predict(X_test)
score = metrics.accuracy_score(y_test,predict)
print("Accuracy: %0.3f" % score)
cm = metrics.confusion_matrix(y_test,predict)
plot_confusion_matrix(cm,classes=['Fake Data','Real Data'])

C:\Users\Ravi\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:152: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)
C:\Users\Ravi\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:152: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


Accuracy: 0.916


NameError: name 'plot_confusion_matrix' is not defined

In [29]:
#Multinomial NB With Hyperparameter
classifier = MultinomialNB(alpha=0.1)

In [30]:
previous_score=0
for alpha in np.arange(0,1,0.1):
    sub_classifier = MultinomialNB(alpha=alpha)
    sub_classifier.fit(X_train,y_train)
    y_predict = sub_classifier.predict(X_test)
    score = metrics.accuracy_score(y_test,y_predict)
    
    if score>previous_score:
        classifier=sub_classifier
    print("Alpha: {}, Score: {}".format(alpha,score))

C:\Users\Ravi\Anaconda3\lib\site-packages\sklearn\naive_bayes.py:480: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)


Alpha: 0.0, Score: 0.8742344706911636
Alpha: 0.1, Score: 0.8834208223972003
Alpha: 0.2, Score: 0.884076990376203
Alpha: 0.30000000000000004, Score: 0.884514435695538
Alpha: 0.4, Score: 0.8862642169728784
Alpha: 0.5, Score: 0.886920384951881
Alpha: 0.6000000000000001, Score: 0.886482939632546
Alpha: 0.7000000000000001, Score: 0.8867016622922135
Alpha: 0.8, Score: 0.8867016622922135
Alpha: 0.9, Score: 0.8853893263342082


In [31]:
classifier.coef_[0]

array([-7.14270532, -8.72280486, -9.465251  , ..., -9.99373388,
       -8.38350869, -9.19407862])

In [32]:
feature_names = cv.get_feature_names()

In [33]:
#Real Data Lowest Negative Numbers
sorted(zip(classifier.coef_[0],feature_names),reverse=True)[:10]

[(-4.937211328430654, 'hillari'),
 (-4.9484007060030155, 'trump'),
 (-5.04667249160301, 'clinton'),
 (-5.382873014842664, 'elect'),
 (-5.731640368853562, 'video'),
 (-5.777651670349984, 'comment'),
 (-5.78592408110518, 'war'),
 (-5.791309920013293, 'us'),
 (-5.852948465509575, 'hillari clinton'),
 (-5.864473737101671, 'vote')]

In [34]:
#Fake Data Highest Negative Numbers
sorted(zip(classifier.coef_[0],feature_names))[:10]

[(-9.993733882499757, '2016 breitbart'),
 (-9.993733882499757, '2016 new york'),
 (-9.993733882499757, '2018 breitbart'),
 (-9.993733882499757, '61'),
 (-9.993733882499757, '78'),
 (-9.993733882499757, '84'),
 (-9.993733882499757, '85'),
 (-9.993733882499757, '87 new'),
 (-9.993733882499757, '87 new york'),
 (-9.993733882499757, '94')]

In [35]:
#Hashing Vectorizer
hs_vectorizer = HashingVectorizer(n_features=5000,non_negative=True)
X = hs_vectorizer.fit_transform(corpus1).toarray()

C:\Users\Ravi\Anaconda3\lib\site-packages\sklearn\feature_extraction\hashing.py:102: DeprecationWarning: the option non_negative=True has been deprecated in 0.19 and will be removed in version 0.21.
  " in version 0.21.", DeprecationWarning)
C:\Users\Ravi\Anaconda3\lib\site-packages\sklearn\feature_extraction\hashing.py:102: DeprecationWarning: the option non_negative=True has been deprecated in 0.19 and will be removed in version 0.21.
  " in version 0.21.", DeprecationWarning)


In [36]:
X.shape

(18285, 5000)

In [37]:
X

array([[0.        , 0.31622777, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [38]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25,random_state=0)

In [39]:
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()
classifier.fit(X_train,y_train)
predict = classifier.predict(X_test)
score = metrics.accuracy_score(y_test,predict)
print("Accuracy: %0.3f" % score)
cm = metrics.confusion_matrix(y_test,predict)
plot_confusion_matrix(cm,classes=['Fake Data','Real Data'])

Accuracy: 0.873


NameError: name 'plot_confusion_matrix' is not defined